In [1]:
%load_ext autoreload
%autoreload 2

from config.rnn import default
from models import RNN
import numpy as np
from functional import seq
from multiprocessing import Process, Pipe
import tensorflow as tf
from tensorpack import (TrainConfig, SyncMultiGPUTrainerParameterServer as Trainer, 
                        PredictConfig, SimpleDatasetPredictor as Predictor,
                        SaverRestore, logger)
from tensorpack.callbacks import (ScheduledHyperParamSetter, MaxSaver, ModelSaver,
                                  DataParallelInferenceRunner as InfRunner)
from tensorpack.predict import SimpleDatasetPredictor
from tensorpack.tfutils.common import get_default_sess_config
from utils import DataManager
from utils.validation import (Accumulator, AggregateMetric, calcu_metrics)

log_dir = './train_log/threshold'
model_loc = './train_log/transfer/all_stages-g2-hidden/all-stages-max-micro-auc.tfmodel'
logger.set_logger_dir(log_dir, action='d')

[1029 16:33:37 @logger.py:74] Argv: /home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /run/user/1012/jupyter/kernel-6fd674f7-7a06-4ae4-a69d-0c728e4e7bc0.json


In [2]:
config = default
config.stages = [2, 3, 4, 5, 6]
config.proportion = {'train': 0.55, 'val': 0.0, 'test': 0.45}
config.annotation_number = None
dm = DataManager.from_config(config)
train_set = dm.get_train_set()
test_set = dm.get_test_set()

In [6]:
config.proportion = {'train': 0.8, 'val': 0.2, 'test': 0.0}
config.annotation_number = 30
config.use_hidden_dense = True
dm = DataManager.from_dataset(train_set, test_set, config)
test_data = dm.get_test_stream()
val_data = dm.get_validation_stream()
dm.get_num_info()

{'test': (4049, 17464), 'train': (3583, 16164), 'val': (829, 3651)}

In [9]:
dm.get_imbalance_ratio()

,train,val,test
ventral nerve cord,8.657682,8.314607,8.803874
anterior midgut primordium,8.478836,8.528736,8.924020
head mesoderm primordium P2,16.478049,14.942308,17.075893
trunk mesoderm primordium,6.823144,6.271930,7.081836
posterior endoderm primordium P2,19.953216,20.815789,21.620112
cellular blastoderm,9.538235,9.109756,9.655263
embryonic foregut,21.117284,21.405405,21.875706
procephalic ectoderm primordium,21.677215,20.815789,23.391566
dorsal prothoracic pharyngeal muscle,19.474286,19.219512,20.652406
posterior endoderm primordium,15.981043,14.942308,16.915929


In [7]:
def run_a_threshold(config, threshold, test_set, pipe):
    model = RNN(config, is_finetuning=False)
    tf.reset_default_graph()
    pred_config = PredictConfig(model=model,
                                session_init=SaverRestore(
                                    model_path=model_loc),
                                output_names=['logits_export', 'label'],
                                )
    # pred = Predictor(pred_config, test_data, nr_proc=2, ordered=False)
    pred = Predictor(pred_config, test_set)

    accumulator = seq(pred.get_result()) \
        .smap(lambda a, b: (a.shape[0], calcu_metrics(a, b, config.validation_metrics, threshold))) \
        .aggregate(Accumulator(*config.validation_metrics), lambda accu, args: accu.feed(args[0], *args[1]))
    metrics = accumulator.retrive()
    pipe.send(metrics)

In [8]:
threshold_choices = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
metrics = []
for thrsd in threshold_choices:
    receiver, sender = Pipe(duplex=False)
    child = Process(target=run_a_threshold, args=(config, thrsd, val_data, sender))
    child.start()
    child.join()
    metric = receiver.recv()
    print(metric)
    metrics.append(metric)

INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
[1029 16:39:56 @sessinit.py:116] Restoring checkpoint from ./train_log/transfer/all_stages-g2-hidden/all-stages-max-micro-auc.tfmodel ...
INFO:tensorflow:Restoring parameters from ./train_log/transfer/all_stages-g2-hidden/all-stages-max-micro-auc.tfmodel


100%|##########|12/12[00:23<00:00, 0.56it/s]


{'mean_average_precision': 0.22488354006913916, 'macro_auc': 0.50243725815210827, 'micro_auc': 0.4987273486775527, 'macro_f1': 0.14699477439479788, 'micro_f1': 0.1560298242225405, 'ranking_mean_average_precision': 0.17020487311383015, 'coverage': 20.30078125, 'ranking_loss': 0.53728818538606027, 'one_error': 0.90364583333333337}
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Scale of 0 disables regularizer.
[1029 16:40:30 @sessinit.py:116]

KeyboardInterrupt: 

Process Process-5:
Traceback (most recent call last):
  File "/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-7-f77fe222dfc1>", line 10, in run_a_threshold
    pred = Predictor(pred_config, test_set)
  File "/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/tensorpack/predict/dataset.py", line 66, in __init__
    self.predictor = OfflinePredictor(config)
  File "/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/tensorpack/predict/base.py", line 168, in __init__
    config.session_init._run_init(sess)
  File "/home/fuxiaofeng/Applications/anaconda3/lib/python3.6/site-packages/tensorpack/tfutils/sessinit.py", line 117, in _run_init
    self.saver.restore(sess, self.path)
  File "/home/fuxiaofeng/Application